### You can also run the notebook in [COLAB](https://colab.research.google.com/github/deepmipt/DeepPavlov/blob/master/examples/trippy_rasa_tutorial.ipynb).

## Rasa 2.X with TripPy

This is a tiny demo to show you how you can build a goal oriented bot in DeepPavlov leveraging the RASA format and the TripPy model in ~<10 lines of code!

So without further ado, let's go!

In [ ]:
!git clone -b rulebased_gobot_trippy https://github.com/Muennighoff/DeepPavlov
%cd DeepPavlov
!pip install -r requirements.txt

Cloning into 'DeepPavlov'...
remote: Enumerating objects: 58548, done.
remote: Counting objects: 100% (1491/1491), done.
remote: Compressing objects: 100% (492/492), done.
remote: Total 58548 (delta 1133), reused 1284 (delta 986), pack-reused 57057
Receiving objects: 100% (58548/58548), 37.81 MiB | 32.11 MiB/s, done.
Resolving deltas: 100% (44978/44978), done.
/content/DeepPavlov
     |████████████████████████████████| 40 kB 27 kB/s 
     |████████████████████████████████| 2.1 MB 14.2 MB/s 
     |████████████████████████████████| 43 kB 2.2 MB/s 
     |████████████████████████████████| 2.9 MB 57.3 MB/s 
     |████████████████████████████████| 1.5 MB 54.9 MB/s 
     |████████████████████████████████| 20.1 MB 1.4 MB/s 
     |████████████████████████████████| 10.4 MB 48.2 MB/s 
     |████████████████████████████████| 510 kB 56.6 MB/s 
     |████████████████████████████████| 7.3 MB 30.8 MB/s 
     |████████████████████████████████| 46 kB 3.5 MB/s 
     |████████████████████████████████| 8.2

In [ ]:
STORIES_PATH="stories.yml"
NLU_PATH="nlu.yml"
DOMAIN_PATH="domain.yml"

### `Stories.yml`

In [ ]:
%%writefile {STORIES_PATH}

stories:
- story: bla bla
  steps:
  - intent: user_hey
  - action: system_hey
  - intent: user_howru
  - action: system_fine
  - intent: user_bye
  - action: system_bye
- story: heybye
  steps:
  - intent: user_hey
  - action: system_hey
  - intent: user_bye
  - action: system_bye
- story: howru
  steps:
  - intent: user_howru
  - action: system_fine


Writing stories.yml


In [ ]:
!cp {STORIES_PATH} stories-trn.yml
!cp {STORIES_PATH} stories-tst.yml 
!cp {STORIES_PATH} stories-val.yml

### `NLU.yml`

In [ ]:
%%writefile {NLU_PATH}
version: "2.0"
dp_version: "2.0"

nlu:
  # regex intent
  - regex: user_hey
    examples: |
      - hey
      - (hello|hi)

  - intent: user_howru
    examples: |
      - how are you
      - how u doin?
  
  # DeepPavlov IntentCatcher examples
  - intent: user_bye
    examples: | 
      - farewell
    regex_examples: |
      - (((good)|(bye-)){0,1}bye)|(see you (later){0,1})


Writing nlu.yml


###`Domain.yml`

In [ ]:
%%writefile {DOMAIN_PATH}
actions:
- system_fine
- system_hey
- system_bye

intents:
- user_hey
- user_bye
- user_howru

responses:
  system_fine:
  - text: im fine
  system_hey:
  - text: hey you!
  system_bye:
  - text: bye!

Writing domain.yml


In [ ]:
%%writefile deeppavlov/configs/go_bot/trippy_md_yaml_minimal.json
{
    "dataset_reader": {
      "class_name": "md_yaml_dialogs_reader",
      "data_path": "{DATA_PATH}",
      "fmt": "yml"
    },
    "dataset_iterator": {
      "class_name": "md_yaml_dialogs_iterator"
    },
    "chainer": {
      "in": ["x"],
      "in_y": ["y"],
      "out": ["y_predicted"],
      "pipe": [
        {
            "class_name": "trippy",
            "in": ["x"],
            "in_y": ["y"],
            "out": ["y_predicted"],
            "load_path": "{MODEL_PATH}/model",
            "save_path": "{MODEL_PATH}/model",
            "pretrained_bert": "bert-base-uncased",
            "max_seq_length": 180,
            "debug": false,
            "database": null,
            "nlg_manager": {
                "class_name": "gobot_json_nlg_manager",
                "data_path": "{DATA_PATH}",
                "dataset_reader_class": "md_yaml_dialogs_reader",
                "actions2slots_path": "{DATA_PATH}/dstc2-actions2slots.json",
                "api_call_action": null
            },
            "api_call_action": null,
            "optimizer_parameters": {"lr": 1e-5, "eps": 1e-6}
        }
      ]
    },
    "train": {
        "batch_size": 32,
        "max_batches": 600,
    
        "metrics": ["per_item_dialog_accuracy"],
        "validation_patience": 10,
        "val_every_n_batches": 15,
        "val_every_n_epochs": -1,
    
        "log_every_n_batches": 15,
        "log_on_k_batches": 10,
    
        "validate_first": true,
    
        "show_examples": false,
        "evaluation_targets": [
          "valid",
          "test"
        ]
      },
    "metadata": {
      "variables": {
        "ROOT_PATH": "~/.deeppavlov",
        "CONFIGS_PATH": "{DEEPPAVLOV_PATH}/configs",
        "DOWNLOADS_PATH": "{ROOT_PATH}/downloads",
        "DATA_PATH": ".",
        "MODELS_PATH": "{ROOT_PATH}/models",
        "MODEL_PATH": "."
      }
    }
  }
  

Overwriting deeppavlov/configs/go_bot/trippy_md_yaml_minimal.json


In [ ]:
!python -m deeppavlov install deeppavlov/configs/go_bot/trippy_md_yaml_minimal.json

     |████████████████████████████████| 641 kB 8.2 MB/s 
     |████████████████████████████████| 5.6 MB 10.7 MB/s 
     |████████████████████████████████| 1.2 MB 56.2 MB/s 


In [ ]:
from deeppavlov import configs
from deeppavlov.core.common.file import read_json

gobot_config = read_json(configs.go_bot.trippy_md_yaml_minimal)

In [ ]:
from deeppavlov import train_model

train_model(gobot_config, download=True)

In [ ]:
from deeppavlov import build_model
bot = build_model(gobot_config)

In [ ]:
bot.reset()
bot(["hello"])

[['hey you!']]

In [ ]:
bot(["how r u?"])

[['im fine']]

In [ ]:
bot(["ok goodbye"])

[['bye!']]